# Deep dive into the Zarr format: Inside `Sentinel_2_L2A.zarr`

## Introduction
This tutorial introduces the structure of a Zarr sample for **Sentinel 2 L2A** data.<br>
We will demonstrate how to visualise the `zarr` encoding structure, explore embedded information, and retrieve metadata for further processing.

### Prerequisites
A sample dataset for this tutorial can be obtained from the [EOPF available Samples](https://common.s3.sbg.perf.cloud.ovh.net/product.html). If further data sets want to be explored, the code indicates where the changes should be addressed.

For local **Sentinel 2 L2A** data exploration, the resource with the format `S02MSIL2A_....zarr` should be located and downloaded in the ame directory as this example.

> **Note:** <br>
> Further sample descriptions will be included in subsequent notebook updates.<br>
> To look into the `.zarr` products naming, visit [the EOPF product types and file naming rules](https://cpm.pages.eopf.copernicus.eu/eopf-cpm/main/PSFD/3-product-types-naming-rules.html).<br>
<br>
> Names, can give some context of the type of product we are working with.
><br>

To manage the indicated libraries, it is recommended to work within a dedicated and stable set up. To ensure package compatibility and avoid conflicts, the following virtual environment setup is suggested:

For Conda:

`conda create --name zarr_explore python=3.11 os xarray zarr numpy jupyter`

For pip:

`python -m venv .zarr_explore`<br>
`.zarr_explore\Scripts\activate.bat`<br>
`pip install os xarray zarr numpy jupyter`

### Setting up the environment
The `xarray` library facilitates the handling of labeled multi-dimensional arrays, enabling an efficient processing. This library and its capabilities for `.zarr` will be explored in along [Chapter 3]().<br>
Check out their [documentation](https://docs.xarray.dev/en/stable/) for additional resources.

We then import the specific dependencies.

In [1]:
import os
import xarray as xr

To allow us retrieve the names for each of the stored groups inside `zarr`, the subsequent function definition allows us looping and retrieving the names to be visualised at each main node. 
This will allow general overview of the elements stored within them without the fine default description.

In [2]:
def print_gen_structure(node, indent=""):
    print(f"{indent}{node.name}")     #allows us access each node
    for child_name, child_node in node.children.items(): #loops inside the selected nodes to extract naming
        print_gen_structure(child_node, indent + "  ") # prints the name of the selected nodes

From `xarray`, The `.open_datatree()` function enables access and decoding of a `DataTree` from a file-like object (in this case, the `.zarr` stored), creating a tree node for each group within the file.

In [3]:
# Open the Zarr store with xarray as a DataTree
s2l2a_zarr_sample= xr.open_datatree(
    "S02MSIL2A_20230629T120347_0000_A064_TC64.zarr",  # Substitute with the downloaded sample of your interest
    engine="zarr", # storage format
    chunks={}, # allows to open the default chunking
)

The following output displays the information contained inside the attributes, conditions, measurements, and quality main `.zarr` groups.

In [4]:
print("Zarr Sentinel 2 L2A Structure")
print_gen_structure(s2l2a_zarr_sample.root) 
print("-" * 30)

Zarr Sentinel 2 L2A Structure
None
  conditions
    geometry
    mask
      detector_footprint
        r10m
        r20m
        r60m
      l1c_classification
        r60m
      l2a_classification
        r20m
        r60m
    meteorology
      cams
      ecmwf
  measurements
    reflectance
      r10m
      r20m
      r60m
  quality
    atmosphere
      r10m
      r20m
      r60m
    l2a_quicklook
      r10m
      r20m
      r60m
    mask
      r10m
      r20m
      r60m
    probability
      r20m
------------------------------


To have a finer visualisation of the `zarr` element, `xarray` also allows us to access a representation of the entire data content within the `.zarr` object. This visualisation displays each group defined inside the `.zarr` file and its respective arrays, including detailed information such as general metadata, dimensions, chunking geometry, and chunk size.

In [5]:
# Open the Zarr store with xarray and print the detailed structure.
# Run this lines in case the print() of the whole data set is of your interest.
# print("Dataset Structure:")
# print(s2l2a_zarr_sample)
# print("-" * 30)

If we are  looking forward to extract specific information from a group, `xarray`'s lables allows us to retrieve by group, the information we are interested in.<br>
<br>
Lets say we are willing to visualise only the `reflectance` values inside this asset.<br>
We need to remember then, that we have 3 available resolutions (10m, 20m and 60m), and by following the `measurements/reflectance` path inside the `zarr`, we can retrieve the `reflectance` group structure:

In [6]:
# Retrieving the reflectance groups:
print(s2l2a_zarr_sample["measurements/reflectance"])

<xarray.DataTree 'reflectance'>
Group: /measurements/reflectance
├── Group: /measurements/reflectance/r10m
│       Dimensions:  (y: 10980, x: 10980)
│       Coordinates:
│         * x        (x) int32 44kB 300005 300015 300025 300035 ... 409775 409785 409795
│         * y        (y) int32 44kB 4600015 4600005 4599995 ... 4490245 4490235 4490225
│       Data variables:
│           b02      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b03      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b04      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b08      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
├── Group: /measurements/reflectance/r20m
│       Dimensions:  (y: 5490, x: 5490)
│       Coordinates:
│         * x        (x) int32 22kB 300010 300030 300050 300070 ... 409750 409770 409790
│         * y        (y) int32 22kB 4600010 4599990 4599970 .

Inside this element, we are able to visualise the three main groups (nodes of the parent node `reflectance`) as `r10`,`r20` and `r60`.<br>
<br>
It is important to point out that if we are willing to actually explore the groups and definition inside the `zarr`, we are able to take out the `print()` statement. <br>
This will enable the `xarray.DataTree` **Drop down** interface that will let us explore interactiveley group related metadata and information. <br>
If we revise further inside each of them we will find the chunks containing the `arrays` with the reflectance information and their corresponding `dtype`.


In [ ]:
# Retrieving the same group in an interactve xarray.DataTree:
s2l2a_zarr_sample["measurements/reflectance"]

<xarray.DataTree 'reflectance'>
Group: /measurements/reflectance
├── Group: /measurements/reflectance/r10m
│       Dimensions:  (y: 10980, x: 10980)
│       Coordinates:
│         * x        (x) int32 44kB 300005 300015 300025 300035 ... 409775 409785 409795
│         * y        (y) int32 44kB 4600015 4600005 4599995 ... 4490245 4490235 4490225
│       Data variables:
│           b02      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b03      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b04      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b08      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
├── Group: /measurements/reflectance/r20m
│       Dimensions:  (y: 5490, x: 5490)
│       Coordinates:
│         * x        (x) int32 22kB 300010 300030 300050 300070 ... 409750 409770 409790
│         * y        (y) int32 22kB 4600010 4599990 4599970 ... 4490270 4490250 4490230
│       Data variables:
│           b01      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b02      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b03      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b04      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b05      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b06      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b07      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b11      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b12      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b8a      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
└── Group: /measurements/reflectance/r60m
        Dimensions:  (y: 1830, x: 1830)
        Coordinates:
          * x        (x) int32 7kB 300030 300090 300150 300210 ... 409650 409710 409770
          * y        (y) int32 7kB 4599990 4599930 4599870 ... 4490370 4490310 4490250
        Data variables:
            b01      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b02      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b03      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b04      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b05      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b06      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b07      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b09      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b11      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b12      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b8a      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>

Through `s2l2a_zarr_sample.attrs[]` we are able to visualise both the `stac_discovery` and `other_metadata`. <br>
<br>
For the properties inside `stac_discovery` for example:

In [8]:
# STAC metadata style:
s2l2a_zarr_sample.attrs["stac_discovery"]['properties']

{'created': '2023-06-29T12:03:47+00:00',
 'datetime': '2018-08-20T08:36:01.024Z',
 'end_datetime': '2018-08-20T08:36:01.024000+00:00',
 'eo:bands': [{'center_wavelength': 442.7,
   'common_name': 'coastal',
   'full_width_half_max': 0.02,
   'name': 'b01',
   'solar_illumination': 1884.69},
  {'center_wavelength': 492.7,
   'common_name': 'blue',
   'full_width_half_max': 0.065,
   'name': 'b02',
   'solar_illumination': 1959.66},
  {'center_wavelength': 559.8,
   'common_name': 'green',
   'full_width_half_max': 0.035,
   'name': 'b03',
   'solar_illumination': 1823.24},
  {'center_wavelength': 664.6,
   'common_name': 'red',
   'full_width_half_max': 0.03,
   'name': 'b04',
   'solar_illumination': 1512.06},
  {'center_wavelength': 704.1,
   'common_name': 'rededge',
   'full_width_half_max': 0.015,
   'name': 'b05',
   'solar_illumination': 1424.64},
  {'center_wavelength': 740.5,
   'common_name': 'rededge',
   'full_width_half_max': 0.015,
   'name': 'b06',
   'solar_illumination'

And from `other_metadata`, we are able to retrieve the information specific to a band, for example the **red** reflectance band `b04`:

In [10]:
# Complementing metadata:
s2l2a_zarr_sample.attrs["other_metadata"]['band_description']['b04']

{'bandwidth': 30.0,
 'central_wavelength': 664.6,
 'onboard_compression_rate': '2.97',
 'onboard_integration_time': '1.3872929',
 'physical_gain': '4.51587741',
 'spectral_response_step': '1',
 'spectral_response_values': '0.00141521 0.02590238 0.11651178 0.39088616 0.74959342 0.94485805 0.98011173 0.99406309 1 0.99545475 0.99052772 0.97733476 0.94055988 0.87894956 0.81629384 0.77345952 0.75448766 0.75991531 0.7826343 0.8101689 0.83612975 0.86125424 0.88609106 0.91138767 0.93405146 0.95042063 0.9592573 0.96039555 0.95913395 0.95809013 0.95527459 0.94376465 0.89490799 0.74426308 0.476777 0.22960399 0.08009118 0.02617076 0.00415242',
 'units': 'nm',
 'wavelength_max': 684.0,
 'wavelength_min': 646.0}

## Conclusion
This tutorial provides an initial understanding of the `zarr` structure for a Sentinel 2 L2A sample. <br>
<br>
By using the `xarray` library, we can effectively navigate and inspect the different components within the `zarr` format, including its metadata and array organisation. 
This foundation will help deeply undestand the subsequent data analysis and processing workflows intended in our series.

For a deeper description of the metadata structure, follow the [metadata structure]() tutorial.